<a href="https://colab.research.google.com/github/markbozsoki/ColabNotebooks/blob/main/Animal_Shelter_Analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Animal Shelter Analytics in Austin, TX

In [1]:
# IMPORTS

import pandas as pd


In [2]:
# DATA ACCESS

intakes = pd.read_csv("Austin_Animal_Center_Intakes.csv")
outcomes = pd.read_csv("Austin_Animal_Center_Outcomes.csv")
stray_map = pd.read_csv("Austin_Animal_Center_Stray_Map.csv")

In [7]:
stray_map

,Animal ID,Found Location,At AAC,Intake Date,Type,Looks Like,Color,Sex,Age,Image Link
0,A814119,11800 GREEN GROVE DRIVE\nAUSTIN 78725\n(30.221...,Yes (come to the shelter),02/19/2021,Dog,Pit Bull,Brown Brindle,Neutered Male,3 years,http://www.petharbor.com/pet.asp?uaid=ASTN.A81...
1,A829667,9807 MARLBOROUGH DRIVE\nAUSTIN 78753\n(30.3573...,Yes (come to the shelter),02/20/2021,Dog,Anatolian Shepherd Mix,Yellow,Intact Male,2 years,http://www.petharbor.com/pet.asp?uaid=ASTN.A82...
2,A829669,12501 TECH RIDGE BOULEVARD\nAUSTIN 78753\n(30....,Yes (come to the shelter),02/20/2021,Dog,Labrador Retriever Mix,Tan/White,Spayed Female,2 years,http://www.petharbor.com/pet.asp?uaid=ASTN.A82...
3,A829713,10605 OLD LOCKHART ROAD\nAUSTIN 78747\n(30.128...,Yes (come to the shelter),02/21/2021,Dog,Doberman Pinsch,Black/Tan,Intact Female,6 months,http://www.petharbor.com/pet.asp?uaid=ASTN.A82...
4,A829659,"14204 JACOBSON ROAD\nAUSTIN 78617\n(30.137238,...",Yes (come to the shelter),02/19/2021,Dog,Labrador Retriever,Black/White,Intact Female,5 months,http://www.petharbor.com/pet.asp?uaid=ASTN.A82...
5,A829660,"14204 JACOBSON ROAD\nAUSTIN 78617\n(30.137238,...",Yes (come to the shelter),02/19/2021,Dog,Carolina Dog,Yellow Brindle/White,Intact Female,5 months,http://www.petharbor.com/pet.asp?uaid=ASTN.A82...
6,A829661,"9604 CARSON CREEK\nAUSTIN 78617\n(30.224098, -...",Yes (come to the shelter),02/19/2021,Cat,Domestic Shorthair,Orange Tabby,Intact Male,1 year,http://www.petharbor.com/pet.asp?uaid=ASTN.A82...
7,A829664,"DEL VALLE AUSTIN 78719\n(30.143087, -97.673385)",Yes (come to the shelter),02/20/2021,Dog,Labrador Retriever,Brown/Black,Intact Male,2 years,http://www.petharbor.com/pet.asp?uaid=ASTN.A82...
8,A829672,"AUSTIN 78744\n(30.186349, -97.738673)",Yes (come to the shelter),02/20/2021,Dog,Doberman Pinsch,Black/Tan,Intact Female,6 months,http://www.petharbor.com/pet.asp?uaid=ASTN.A82...
9,A716730,"4434 FRONTIER TRAIL\nAUSTIN 78745\n(30.23124, ...",Yes (come to the shelter),02/22/2021,Cat,Domestic Shorthair Mix,Brown Tabby,Spayed Female,5 years,http://www.petharbor.com/pet.asp?uaid=ASTN.A71...
